In [4]:
import pandas as pd

In [7]:
# forest = pd.read_csv("models_hypotheses/Random_Forest_gpu_hypotheses_as_sentences.csv")
xgb = pd.read_csv("models_hypotheses/XGBoost_hypotheses_as_sentences.csv")
# rules = pd.read_csv("models_hypotheses/rules_df.csv")
lgb = pd.read_csv("models_hypotheses/LightGBM_hypotheses_as_sentences.csv")
lift = pd.read_csv("models_hypotheses/LIFT_hypotheses_as_sentences.csv")
lift.rename(columns={'Cancer Type': 'cancer_type'}, inplace=True)

In [19]:
# Define source-to-method mapping
sources = {
    'lgb': 'LGBM',
    'xgb': 'XGBS',
    'lift': 'LIFT',
    # 'rules': 'RULE',
    # 'forest': 'FRST',
}

# Initialize an empty list to store processed dataframes
processed_dfs = []

# Process each dataframe in a loop
for df_name, df in [('xgb', xgb), ('lgb', lgb), ('lift', lift)]:
    # Create a copy to avoid modifying the original
    temp_df = df.copy()

    # Add method column
    temp_df['method'] = sources[df_name]

    # Rename the hypothesis column to 'hypo' as requested in output
    temp_df = temp_df.rename(columns={'hypothesis': 'hypo_factors'})

    # Add to list of processed dataframes
    processed_dfs.append(temp_df)

# Combine all processed dataframes
combined_df = pd.concat(processed_dfs, ignore_index=True)

combined_df.sort_values(by=['method', 'support'], ascending=[True, False], inplace=True)
combined_df['rank'] = combined_df.groupby('method')['support'].rank(ascending=False, method='first').astype(int)
combined_df['hypo_id'] = combined_df.apply(
        lambda row: f"{sources[df_name]}.{row['cancer_type'].upper().replace(' ', '_')}.{row['rank']}",
        axis=1
    )

# Display the first few rows to verify the result
combined_df.head()

,Codons,Hugo_Symbol,Start_Position,cancer_type,upstream_gene_variant,TMB (nonsynonymous),End_Position,Site1_Hugo_Symbol,Sex,Event_Info,...,splice_acceptor_variant,support,hypo_factors,method,VAR_TYPE_SX,Unnamed: 0,Consequence,Position,rank,hypo_id
640,NaN,NaN,NaN,Colorectal Carcinoma,NaN,8.766667,NaN,NaN,Male,NaN,...,NaN,262.0,TMB (nonsynonymous) value is 8.766666667 AND S...,LGBM,NaN,NaN,NaN,NaN,1,LIFT.COLORECTAL_CARCINOMA.1
641,NaN,NaN,NaN,Colorectal Carcinoma,NaN,3.333333,NaN,NaN,Female,NaN,...,NaN,105.0,TMB (nonsynonymous) value is 3.333333333 AND S...,LGBM,NaN,NaN,NaN,NaN,2,LIFT.COLORECTAL_CARCINOMA.2
767,NaN,NaN,NaN,Gastric Cancer,NaN,2.900000,NaN,NaN,Male,NaN,...,NaN,95.0,TMB (nonsynonymous) value is 2.9 AND Sex value...,LGBM,NaN,NaN,NaN,NaN,3,LIFT.GASTRIC_CANCER.3
642,NaN,NaN,NaN,Colorectal Carcinoma,NaN,3.100000,NaN,NaN,Female,NaN,...,NaN,94.0,TMB (nonsynonymous) value is 3.1 AND Sex value...,LGBM,NaN,NaN,NaN,NaN,4,LIFT.COLORECTAL_CARCINOMA.4
768,NaN,NaN,NaN,Gastric Cancer,NaN,2.833333,NaN,NaN,Male,NaN,...,NaN,85.0,TMB (nonsynonymous) value is 2.833333333 AND S...,LGBM,NaN,NaN,NaN,NaN,5,LIFT.GASTRIC_CANCER.5


In [20]:
# make the first column the hypo_id, the second the cancer_type, the third the hypo_factors while keeping all the other columns
combined_df = combined_df[['hypo_id', 'cancer_type', 'hypo_factors'] + [col for col in combined_df.columns if col not in ['hypo_id', 'cancer_type', 'hypo_factors']]]


In [21]:
combined_df.shape

(1295, 29)

In [22]:
combined_df['method'].value_counts()

method
LGBM    634
XGBS    621
LIFT     40
Name: count, dtype: int64

In [23]:
combined_df.to_csv("models_hypotheses/combined_hypotheses.csv", index=False)